In [4]:
from chengyu.config import settings
from chengyu.dedupe import list_existing_chengyu
from chengyu.gen import gen_unique_episode_strict, script_to_markdown
#from chengyu.cover import draw_cover_png
from chengyu.tts import tts_mp3
from chengyu.publisher import publish_episode
from chengyu.cover_ai import draw_cover_png_story
# ...
# cover_png = draw_cover_png(settings.SHOW_NAME, data["chengyu"], data["pinyin"], data["gloss"])
cover_png = draw_cover_png_story(
    chengyu=data["chengyu"],
    pinyin=data["pinyin"],
    english=data["gloss"],
    script=data["script"],
    show_mark=settings.SHOW_NAME,
    model=settings.IMAGE_MODEL,
    size=settings.IMAGE_SIZE,
    out_size=3000
)


# 0) Already-used idioms (full set)
forbidden = list_existing_chengyu(settings.REPO, settings.GITHUB_BRANCH)

# 1) Strict-unique generation
data = gen_unique_episode_strict(settings.SHOW_NAME, settings.GEN_MODEL, forbidden,
                                 batch_size=20, max_rounds=20)

# 2) Cover image
cover_png = draw_cover_png(settings.SHOW_NAME, data["chengyu"], data["pinyin"], data["gloss"])

# 3) TTS (disable by setting SKIP_TTS=True)
SKIP_TTS = False
audio_mp3 = None if SKIP_TTS else tts_mp3(data["script"], settings.TTS_MODEL, settings.TTS_VOICE)

# 4) Structured Markdown body
body_md = script_to_markdown(data["chengyu"], data["pinyin"], data["gloss"], data["teaser"], data["script"], settings.GEN_MODEL)

# 5) Publish
publish_episode(
    show_name=settings.SHOW_NAME,
    repo=settings.REPO,
    branch=settings.GITHUB_BRANCH,
    site_url=settings.SITE_URL,
    baseurl=settings.BASEURL,
    publish_time_utc=settings.PUBLISH_TIME_UTC,
    data=data,
    body_md=body_md,
    cover_png=cover_png,
    audio_mp3=audio_mp3,
    dry_run=settings.DRY_RUN,
)


AttributeError: 'Settings' object has no attribute 'IMAGE_MODEL'

In [3]:
from chengyu.cover_ai import draw_cover_png_story
from chengyu.config import settings

png = draw_cover_png_story(
    chengyu="画蛇添足",
    pinyin="huà shé tiān zú",
    english="to add unnecessary details; to overdo something",
    script="Here’s the story behind it: In ancient times ...",  # any short origin helps
    show_mark=settings.SHOW_NAME,
    model=settings.IMAGE_MODEL,
    size=settings.IMAGE_SIZE,          # e.g. "1024x1024" or "auto"
    out_size=3000,
    seal_text="成语",                   # small red stamp
    seal_pos="tr"                      # 'tr', 'tl', 'br', 'bl'
)
open("cover_example.png","wb").write(png)


225719

In [7]:
import os; print(os.path.exists(settings.CN_BRUSH_FONT), settings.CN_BRUSH_FONT)


AttributeError: 'Settings' object has no attribute 'CN_BRUSH_FONT'